### Naive Bayes classifier on MNIST
My previous implementation was like naive bayes, naive. I didnt understand much about distributions and tensor operations so no shock the code didnt work. As a noob, it was hard for me to understand high level implementations of others. So I will try to explain as much as I can.

In [1]:
import gzip,os,pickle,math

def get_data(file_path, file_name):
    with gzip.open(os.path.join(file_path, file_name), 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')

    return x_train, y_train, x_valid, y_valid


dir_ =
filename = 
train_x, train_y, test_x, test_y = get_data(dir_,filename)


Bayes rule says </br>
$P(Y|X) = \frac{P(X|Y)P(Y)}{P(X)}$
which means that the probability that a particular sample belongs to a particular class Y equal what probability that sample has according to the class distribution times the probability of a class in the dataset.

For MNIST, we can say $y = argmax_i(P(Y_i|X))$, where $i$ is the number of classes.

Lets focus on each term of $P(Y_i|X)$
1. $P(X|Y_i)$ : to get this term we formulate a distribution of each class using a multivariate gaussian distribution $\mathcal{N}(\mu,\Sigma)$, where $\mu$ is mean for each pixel and $\Sigma$ is the covariance matrix. Each image has 784 pixels. So our distribution will be of 784 dimensions. Also we assume that *each pixel is independent to other* so our covariance matrix will be a diagonal matrix. In my previous implementation, I was using a univariate normal distribution for each pixel for each class. Using multivariate distribution is much easier. So, for multivariate gaussian $P(X|Y_i)= \frac{1}{\sqrt{ (2\pi)^D |\Sigma| }} exp\left({ -\frac{1}{2}(X – \mu)^T \Sigma^{-1} (X – \mu) }\right)$

 The expression look daunting but we dont have to worry about it. Pytorch will directly give us the probabilites as you will see soon.
 
2. $P(Y)$: This is relatively easy to calculate. It is simply: </br>$\frac{\text{instances of that class}}{\text{total length of train set}}$

3. $P(X)$: The good thing is that we can ignore this term. Since we are calculating argmax and $P(X)$ will be same for all the classes. But if you wanted to calculate it nevertheless, then we can use multivariate gaussian again created using mean and standard deviation of entire train set.

In [2]:
import torch
from torch.distributions.multivariate_normal import MultivariateNormal

class_stats = {}
class_means_std = {}
for x,y in zip(train_x,train_y):
    if y not in class_stats:
        class_stats[y]=[]
        class_means_std[y] ={}
    class_stats[y].append(x)

for i in class_stats:
    class_instances = torch.Tensor(class_stats[i])
    mean = class_instances.mean(dim=0)
    var = class_instances.var(dim=0)
    class_means_std[i]["mean"]= mean
    class_means_std[i]["var"]= var
    d = torch.eye(len(var))
    class_means_std[i]["cov"] = d*(var[None,:]+1e-01)
    class_means_std[i]["dist"] = MultivariateNormal(class_means_std[i]["mean"],class_means_std[i]["cov"])
    class_means_std[i]["prob"] = float(len(class_instances)) / len(train_x)
    

#### Implementation tricks:
Now that we have the mean, variance and distribution for each digits we can finally calculate $P(X|Y)$ but these values can be very small. To avoid that we take log of $P(Y|X)$. Since we are finding argmax and log is monotically increasing this results no problem. So:

$y = argmax_i(\log{P(Y_i|X)}) = argmax_i(\log{P(X|Y)} + \log{P(Y)})$ since we chose to ignore $P(x)$

Note: With continous distributions, we usually get $\log{P(X|Y)}$ using log_prob from distributions. The log_prob is log of the probability distribution function and not the actual probability of that sample(actual probability of a sample from continous distribution is 0 beacuse of infinite samples in the distribution) so its value can be greater than zero. I mentioned because I was perplexed with +ve values for log_probs. It turns out that PDFs give relative likelihood not actual probability.

In [3]:
accurate = 0
for x,y in zip(test_x,test_y):
    max = -math.inf
    index = None
    for i in class_means_std:
        prob = class_means_std[i]["dist"].log_prob(torch.Tensor(x)) + math.log(class_means_std[i]["prob"])
        if prob >max:
            index = i
            max = prob
    if index==y:
        accurate+=1

print("Accuracy {}".format(accurate*100/len(test_x)))

Accuracy 78.32
